In [86]:
from readability.readability import Document, Unparseable
from time import sleep
from zipfile import ZipFile
from tqdm import tqdm
import requests, re, glob, chardet, pandas as pd

In [87]:
glob.glob('aggreg*.tsv')

['aggregators.tsv']

In [88]:
df = pd.read_csv('aggregators.tsv', sep='\t')

In [89]:
df.head()

,link,new_created,new_id,original_id,source,title,topic_created,topic_id,topic_last
0,http://fakty.ua/262212-sudba-savchenko-budet-z...,1521650424,62019456,NaN,ukrnet,"Судьба Савченко будет зависеть от «кино», – п...",1521650424,34151157,1521650424
1,http://ua1.com.ua/politics/bankova-rozkruchue-...,1521647666,62018221,NaN,ukrnet,Банкова розкручує Савченко як кандидата проти ...,1521650424,34151157,1521650424
2,http://hronika.info/politika/304444-savchenko-...,1521647066,62017938,NaN,ukrnet,"Савченко пришла на заседание комитета Рады, с ...",1521650424,34151157,1521650424
3,http://xn--j1aidcn.org/%d1%81%d0%b0%d0%b2%d1%8...,1521646768,62017779,NaN,ukrnet,Савченко отличилась новой скандальной выходкой...,1521650424,34151157,1521650424
4,https://www.segodnya.ua/politics/snyat-nepriko...,1521645745,62017264,NaN,ukrnet,Снять неприкосновенность с Савченко могут завт...,1521650424,34151157,1521650424


## make corpus of htmls

In [5]:
zf = ZipFile('aggr_htmls.zip', 'a')

In [6]:
def get_link(url):
    for _ in range(5):
        try:
            r = requests.get(url)
            if r.status_code == 200:
                encoding = chardet.detect(r.content)
                r.encoding = encoding['encoding']
                return r.text
        except:
            sleep(1)

In [7]:
files = zf.namelist()
for line in tqdm(df.to_dict(orient='records')):
    if f'{line["new_id"]}.html' in files[:-2]:
        continue
    html = get_link(line['link'])
    if not html:
        continue
    try:
        readed_doc = Document(html)
        summary = re.sub('\s+', ' ', readed_doc.summary())
        if len(re.sub('[^А-яІіЄєЇїҐґ]', '', summary)) < 3:
            continue
        with zf.open(f'{line["new_id"]}.html', 'w') as f:
            f.write(summary.encode('utf-8'))
    except Unparseable:
        continue

100%|██████████| 13982/13982 [5:00:28<00:00,  1.29s/it]  


In [ ]:
zf.close()

## tokenize and detect lang

In [90]:
from polyglot.text import Text
from bs4 import BeautifulSoup
from polyglot.detect import Detector # cld2
from operator import itemgetter
import pdb

In [91]:
def get_lang(text):
    '''
    Detects whether text is in Russian or in Ukrainian.
    Accepts string or list of words as input
    Returns None if unsure. 
    '''
    if isinstance(text, list) and all(isinstance(w, str) for w in text):
        text = ' '.join(text)
    elif isinstance(text, str):
        pass
    else:
        raise TypeError(f'{type(text)} provided as input. Please give a str or a list of words')
        
    lang = [{'lang': l.code, 'sure': l.confidence}
            for l in Detector(text, quiet=True).languages
            if l.code in ['uk', 'ru']]
    lang = sorted(lang, key=itemgetter('sure'))
    if len(lang) > 0:
        return lang[-1]['lang']

In [ ]:
with ZipFile('aggr_htmls.zip') as zf:
    htmls_n = zf.namelist()
    with ZipFile('aggr_texts.zip', 'w') as zft:
        for html_f in htmls_n:
            with zf.open(html_f) as f:
                html = f.read().decode()
                html = re.sub('[\n\s]+|https?://.*', ' ', html).strip()
                soup = BeautifulSoup(html, 'lxml')
                [t.extract() for t in soup.select('.')
                 if len(re.sub('[^А-яІіЇїЄєҐґ]', '', t.text)) < 2]
                paragraphs = [p.get_text(' ') for p in soup.select('p, h2, h3, h4, h5, h6')]

                text = '\n\n'.join(['\n'.join([' '.join([str(w) for w in sent.words])
                                               for sent in Text(p).sentences])
                                  for p in paragraphs
                                    if len(p) > 2])
                lang = get_lang(text)
                doc_id = re.sub('\D', '', html_f)
                if len(re.sub('[^А-яІіЇїЄєҐґ]', '', text)) > 10:
                    with zft.open(f'{doc_id}_{lang}.txt', 'w') as f:
                        f.write(text.encode('utf-8'))

In [93]:
with ZipFile('aggr_texts.zip') as zf:
    text_ids = zf.namelist()
    for f in text_ids:
        with zf.open(f) as t:
            print(t.read().decode())
            break

В четверг , 22 марта , в 8.30 комитет по вопросам регламента и организации работы Верховной Рады соберется на заседание , чтобы рассмотреть три представления Генпрокуратуры о привлечении Надежды Савченко к уголовной ответственности , ее задержании и аресте .
После этого судьбу Савченко будут решать народные депутаты на пленарном заседании .

« Нет никаких сомнений , что Надежду Савченко лишат депутатской неприкосновенности » , — сказал « ФАКТАМ » глава правления Центра прикладных политических исследований « Пента » Владимир Фесенко и объяснил почему .

« Ни одна политическая сила в парламенте не поддерживает Савченко .
Она всегда была чужой в Верховной Раде .
К тому же история с проносом в сумке гранат и пистолета в сессионный зал явно сыграла не в ее пользу .
Поэтому народные депутаты , скорее всего , поддержат и представление на задержание Савченко на время следствия .
Что же касается согласия парламента на ее арест , то здесь сложно что - либо прогнозировать .
Все будет зависеть от 

## Construct pairs balanced

In [94]:
from itertools import combinations
from sklearn.model_selection import train_test_split
import numpy as np

In [100]:
# text_ids = [int(re.sub('\D', '', i)) for i in text_ids]
df_with_texts = df.loc[df.new_id.isin(text_ids), ].copy()
pairs = []

for topic in np.unique(df_with_texts.topic_id):
    same = df_with_texts.loc[df_with_texts.topic_id == topic, ].copy()
    
    sample_size = len(same) if len(same) < 15 else 15
    
    same = same.sample(n=sample_size)
    other = df_with_texts.loc[df_with_texts.topic_id != topic, ].copy(
                                         ).sample(n=sample_size)
    
    topic_sample = pd.concat([same, other]
                            ).reindex(columns=['new_id', 'topic_id'])
    
    topic_sample = [(n.new_id, n.topic_id) for i, n in topic_sample.iterrows()]
    
    pairs += [[n1, n2, t1 == t2]
              for (n1, t1), (n2, t2) in combinations(topic_sample, 2)]

In [101]:
pairs = pd.DataFrame(pairs, columns=['id1', 'id2', 'is_similar']
                    ).sample(frac=1)

train, test = train_test_split(pairs,
                               test_size=0.5,
                               random_state=np.random.randint(0, 10000))

In [102]:
train.to_csv('train_pair_ids.tsv', sep='\t', index=False)
test.to_csv('test_pair_ids.tsv', sep='\t', index=False)

In [103]:
test.is_similar.value_counts()

False    53970
True     16305
Name: is_similar, dtype: int64